In [64]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

# Import our input dataset
batting_df = pd.read_csv('../Data/binDATA.csv', index_col=0)
batting_df.head()

,teamID,yearID,G,W,L,H,AB,SB,HR,2B,...,RA,IPouts,HA,BBA,BAVG,SLG,H/BBA,IP,WHIP,Run_Buckets
0,ARI,2015,162,79,83,1494,5649,132,154,289,...,713,4400,1450,500,0.264472,0.414410,1950,1466.666667,1.329545,MID
1,ARI,2016,162,69,93,1479,5665,137,190,285,...,890,4354,1563,603,0.261077,0.431774,2166,1451.333333,1.492421,MID-HIGH
2,ARI,2017,162,93,69,1405,5525,103,220,314,...,659,4323,1309,516,0.254299,0.444706,1825,1441.000000,1.266482,HIGH
3,ARI,2018,162,82,80,1283,5460,79,176,259,...,644,4389,1313,522,0.234982,0.397436,1835,1463.000000,1.254272,LOW-MID
4,ARI,2019,162,85,77,1419,5633,88,220,288,...,743,4395,1400,516,0.251908,0.434404,1916,1465.000000,1.307850,HIGH


In [65]:
grouped_batting_df = batting_df.set_index(['teamID', 'yearID'])
grouped_batting_df

G   W    L     H    AB   SB   HR   2B  3B    R  ...   RA  \
teamID yearID                                                    ...        
ARI    2015    162  79   83  1494  5649  132  154  289  48  720  ...  713   
       2016    162  69   93  1479  5665  137  190  285  56  752  ...  890   
       2017    162  93   69  1405  5525  103  220  314  39  812  ...  659   
       2018    162  82   80  1283  5460   79  176  259  50  693  ...  644   
       2019    162  85   77  1419  5633   88  220  288  40  813  ...  743   
...            ...  ..  ...   ...   ...  ...  ...  ...  ..  ...  ...  ...   
WAS    2017    162  97   65  1477  5553  108  215  311  31  819  ...  672   
       2018    162  82   80  1402  5517  119  191  284  25  771  ...  682   
       2019    162  93   69  1460  5512  116  231  298  27  873  ...  724   
       2021    162  65   97  1388  5385   56  182  272  20  724  ...  820   
       2022    162  55  107  1351  5434   75  136  252  20  603  ...  855   

               IPouts    HA  BBA      BAVG       SLG  H/BBA           IP  \
teamID yearID                                                              
ARI    2015      4400  1450  500  0.264472  0.414410   1950  1466.666667   
       2016      4354  1563  603  0.261077  0.431774   2166  1451.333333   
       2017      4323  1309  516  0.254299  0.444706   1825  1441.000000   
       2018      4389  1313  522  0.234982  0.397436   1835  1463.000000   
       2019      4395  1400  516  0.251908  0.434404   1916  1465.000000   
...               ...   ...  ...       ...       ...    ...          ...   
WAS    2017      4340  1300  495  0.265982  0.449307   1795  1446.666667   
       2018      4338  1320  487  0.254124  0.418525   1807  1446.000000   
       2019      4318  1340  517  0.264877  0.454463   1857  1439.333333   
       2021      4183  1364  548  0.257753  0.417084   1912  1394.333333   
       2022      4235  1469  558  0.248620  0.377438   2027  1411.666667   

                   WHIP  Run_Buckets  
teamID yearID                         
ARI    2015    1.329545          MID  
       2016    1.492421     MID-HIGH  
       2017    1.266482         HIGH  
       2018    1.254272      LOW-MID  
       2019    1.307850         HIGH  
...                 ...          ...  
WAS    2017    1.240783         HIGH  
       2018    1.249654     MID-HIGH  
       2019    1.290181         HIGH  
       2021    1.371265          MID  
       2022    1.435891          LOW  

[210 rows x 21 columns]

In [66]:
# Generate our categorical variable lists
grouped_batting_cat = grouped_batting_df.dtypes[grouped_batting_df.dtypes == "object"].index.tolist()

In [67]:
# Check the number of unique values in each column
grouped_batting_df[grouped_batting_cat].nunique()

Run_Buckets    5
dtype: int64

In [68]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(grouped_batting_df[grouped_batting_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(grouped_batting_cat)
encode_df.head()

c:\Users\Jturn\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,Run_Buckets_HIGH,Run_Buckets_LOW,Run_Buckets_LOW-MID,Run_Buckets_MID,Run_Buckets_MID-HIGH
0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,1.0
2,1.0,0.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0


In [69]:
new_df = batting_df.merge(encode_df,left_index=True, right_index=True)
new_df = new_df.drop(grouped_batting_cat,1)
new_df.head()

C:\Users\Jturn\AppData\Local\Temp\ipykernel_15528\3145326927.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  new_df = new_df.drop(grouped_batting_cat,1)


,teamID,yearID,G,W,L,H,AB,SB,HR,2B,...,BAVG,SLG,H/BBA,IP,WHIP,Run_Buckets_HIGH,Run_Buckets_LOW,Run_Buckets_LOW-MID,Run_Buckets_MID,Run_Buckets_MID-HIGH
0,ARI,2015,162,79,83,1494,5649,132,154,289,...,0.264472,0.414410,1950,1466.666667,1.329545,0.0,0.0,0.0,1.0,0.0
1,ARI,2016,162,69,93,1479,5665,137,190,285,...,0.261077,0.431774,2166,1451.333333,1.492421,0.0,0.0,0.0,0.0,1.0
2,ARI,2017,162,93,69,1405,5525,103,220,314,...,0.254299,0.444706,1825,1441.000000,1.266482,1.0,0.0,0.0,0.0,0.0
3,ARI,2018,162,82,80,1283,5460,79,176,259,...,0.234982,0.397436,1835,1463.000000,1.254272,0.0,0.0,1.0,0.0,0.0
4,ARI,2019,162,85,77,1419,5633,88,220,288,...,0.251908,0.434404,1916,1465.000000,1.307850,1.0,0.0,0.0,0.0,0.0


In [70]:
# Split our preprocessed data into our features and target arrays
X = new_df[["SLG", "SB", "BAVG"]].values
y = new_df[["Run_Buckets_HIGH"]].values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=42)

In [71]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [72]:
# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  8
hidden_nodes_layer2 = 5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 8)                 32        
                                                                 
 dense_10 (Dense)            (None, 5)                 45        
                                                                 
 dense_11 (Dense)            (None, 1)                 6         
                                                                 
Total params: 83 (332.00 Byte)
Trainable params: 83 (332.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [73]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [74]:
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100


6/6 [==============================] - 0s 1ms/step - loss: 0.7511 - accuracy: 0.4643
Epoch 2/100
6/6 [==============================] - 0s 1ms/step - loss: 0.7273 - accuracy: 0.4762
Epoch 3/100
6/6 [==============================] - 0s 1ms/step - loss: 0.7037 - accuracy: 0.5060
Epoch 4/100
6/6 [==============================] - 0s 1ms/step - loss: 0.6820 - accuracy: 0.5417
Epoch 5/100
6/6 [==============================] - 0s 1ms/step - loss: 0.6601 - accuracy: 0.5893
Epoch 6/100
6/6 [==============================] - 0s 2ms/step - loss: 0.6395 - accuracy: 0.6310
Epoch 7/100
6/6 [==============================] - 0s 1ms/step - loss: 0.6198 - accuracy: 0.6964
Epoch 8/100
6/6 [==============================] - 0s 1ms/step - loss: 0.6013 - accuracy: 0.7381
Epoch 9/100
6/6 [==============================] - 0s 2ms/step - loss: 0.5822 - accuracy: 0.7560
Epoch 10/100
6/6 [==============================] - 0s 1ms/step - loss: 0.5647 - accuracy: 0.7738
Epoch 11/100
6/6 [=======================

In [75]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

2/2 - 0s - loss: 0.2717 - accuracy: 0.8095 - 111ms/epoch - 56ms/step
Loss: 0.27174192667007446, Accuracy: 0.8095238208770752
